In [2]:
import os
import re
import requests
from groq import Groq

In [50]:
os.environ["GROQ_API_KEY"] = "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"
model="llama-3.1-70b-versatile"
os.environ["WOLFRAM_ALPHA_APPID"] = "Y89ETU-L7Y6WK73H8"
os.environ["USER_AGENT"] = "myagent"
os.environ["TAVILY_API_KEY"] = "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"
os.environ["ZILLIZ_TOKEN"]="7e59b4419123f4ec6ceaa17aed0c710b9d656484d975aecae74c83d941e28abb0d1908a4cf870ed8fd267fc0ddab51aab1bed9bf"
os.environ["ZILLIZ_URI"]="https://in03-209ae722b58c1e4.api.gcp-us-west1.zillizcloud.com"

## Specification:

1. Store notes in vector database
2. Retrieve initial notes for query
3. Use math tool and web search to begin solving problem while explaining steps
4. Use all these in any order and however many times necessary to answer

### Paul's Online Notes Calculus

https://tutorial.math.lamar.edu/

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

DIRECTORY_PATH = "calculus_notes/"
CHUNK_SIZE = 2000
CHUNK_OVERLAP = 40

loader = PyPDFDirectoryLoader(DIRECTORY_PATH)
pdf_content = loader.load()

In [5]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
docs = text_splitter.split_documents(pdf_content)

In [6]:
from langchain_nomic.embeddings import NomicEmbeddings
embeddings = NomicEmbeddings(model="nomic-embed-text-v1.5", inference_mode='local')

In [7]:
from langchain.vectorstores import Qdrant
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    path="notes/calculus",
    collection_name="course_documents",
    force_recreate=True
)

In [44]:
from langchain_milvus import Zilliz

vectorstore = Zilliz.from_documents(
    documents=docs,
    embedding=embeddings,
    auto_id=True,
    connection_args={
        "token":os.environ.get("MILVUS_TOKEN"),
        "uri":os.environ.get("CLUSTER_ENDPOINT")
    }
)

In [52]:
# from pymilvus import connections
# # Connect to your Zilliz Cloud instance
# connections.connect(
#     alias="default", 
#     uri=os.environ.get("ZILLIZ_URI"),
#     token=os.environ.get("ZILLIZ_TOKEN"),
# )


vectorstore = Zilliz(
    embedding_function=embeddings,
    collection_name="LangChainCollection",
    connection_args={
        "uri":os.environ.get("ZILLIZ_URI"),
        "token":os.environ.get("ZILLIZ_TOKEN"),
    }
)

In [53]:
query = "what is the ratio test?"
found_docs = vectorstore.similarity_search(query)

print(found_docs[0].page_content)

Chapter 10 : Series and Sequences Section 10.10 : Ratio Test
Again, we will run across factorials with parenthesis so don’t drop them. This is often one of the
morecommonmistakesthatstudentsmakewhentheyfirstrunacrossfactorials.
Okay,wearenowreadyforthetest.
Ratio Test
Supposewehavetheseries∑
an. Define,
L=lim
n→∞⏐⏐⏐⏐an+1
an⏐⏐⏐⏐
Then,
1.ifL < 1theseriesisabsolutelyconvergent(andhenceconvergent).
2.ifL > 1theseriesisdivergent.
3.ifL= 1theseriesmaybedivergent,conditionallyconvergent,orabsolutelyconver-
gent.
Aproofofthistestisattheendofthesection.
Noticethatinthecaseof L= 1theratiotestisprettymuchworthlessandwewouldneedtoresort
toadifferenttesttodeterminetheconvergenceoftheseries.
Also, the absolute value bars in the definition of Lare absolutely required. If they are not there it
willbeimpossibleforustogetthecorrectanswer.
Let’stakealookatsomeexamples.
Example 1
Determineifthefollowingseriesisconvergentordivergent.
∞∑
n=1(−10)n
42n+1(n+ 1)
Solution
With this first example let’s be a litt

In [46]:
retriever = vectorstore.as_retriever(k=4)

In [54]:
from langchain.tools import tool

@tool
def wolfram_call(query: str) -> str:
    """Calls the wolfram alpha api on query.

    Args:
        query (str): The query to use as a parameter in the function call. For example, "Solve 3x=5"

    Returns:
        str: Returns the result of the api call on the query.
    """
    params = {
        "input": query,
        "appid": os.environ.get("WOLFRAM_ALPHA_APPID"),
        "format": "plaintext",
        "output": "json",
    }
    response = requests.get("https://api.wolframalpha.com/v2/query", params=params)
    full_response = response.json()
    pods = [x["subpods"] for x in full_response["queryresult"]["pods"]]
    return str(pods)

@tool
def notes_search(query: str) -> str:
    """Calls the vector database of calculus notes to get any definitions, theorems, axioms, proofs, examples, information for summarizing.
    The database contain notes for Calculus 1, 2 and 3. It covers the following topics:
    limits, derivatives, derivative applications, integrals, integral applications, integration techniques, more integral applications, parametric and polar, series and sequences, vectors, 3-d space, partial derivatives, applications of partial derivatives, multiple integrals, line integrals, surface integrals, various calculus proofs, review of trignometry and functions.

    Args:
        query (str): the query sent to the vector database.

    Returns:
        str: Returns the documents retrieved from the vector database based on the query.
    """
    return retriever.invoke(query)

tools = [wolfram_call, notes_search]

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_groq import ChatGroq

llm = ChatGroq(temperature=0, model=model)

system = """You are a mathematical assistant for a course on calculus. 
Create a plan to solve the question and then solve it in a step-by-step manner.
You have 2 tools available. Before using any tool, explain the logic behind using it and the arguments given to the tool.
Use the wolfram tool wherever it is possible to create a valid query to solve any algebra required in the steps of the problem.
Use the notes_search tool as a reference whenever required or if you are unsure about anything to search the course notes for the precise definitions, theorems, axioms, proofs, examples or information for summarizing in calculus 1,2 and 3 and related topics. Explain whatever is retrieved using this tool."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        MessagesPlaceholder("chat_history", optional=True),
        ('human', '{input}'),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [11]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [37]:
result = agent_executor.invoke({"input": "Give a summary of the tests for checking series convergence and divergence"})



> Entering new AgentExecutor chain...

Invoking: `notes_search` with `{'query': 'summary of tests for series convergence and divergence in calculus 3 notes'}`


[Document(metadata={'source': 'calculus_notes/Calculus.pdf', 'page': 805, '_id': 'a78058190b6248a79a5557db74ec69b5', '_collection_name': 'course_documents'}, page_content='Chapter 10 : Series and Sequences Section 10.4 : Convergence & Divergence of Series\nOne of the more common mistakes that students make when they first get into series is to as-\nsume that if lim\nn→∞an= 0then∑anwill converge. There is just no way to guarantee this so be\ncareful!\nLet’stakeaquicklookatanexampleofhowthistestcanbeused.\nExample 5\nDetermineifthefollowingseriesisconvergentordivergent.\n∞∑\nn=04n2−n3\n10 + 2 n3\nSolution\nWith almost every series we’ll be looking at in this chapter the first thing that we should do\nistakealookattheseriestermsandseeiftheygotozeroornot. Ifit’sclearthattheterms\ndon’tgotozerousetheDivergenceTestandbedonewiththep

In [30]:
result = agent_executor.invoke({"input": "What is the chain rule in derivation?"})



> Entering new AgentExecutor chain...

Invoking: `notes_search` with `{'query': 'chain rule derivation'}`


[Document(metadata={'source': 'calculus_notes/Calculus.pdf', 'page': 273, '_id': '0c1dcac30b654926bdcdadcbaaa46c18', '_collection_name': 'course_documents'}, page_content='Chapter 3 : Derivatives Section 3.9 : Chain Rule\nSo, in the first term the outside function is the exponent of 4and the inside function\nis the cosine. In the second term it’s exactly the opposite. In the second term the\noutside function is the cosine and the inside function is t4. Here’s the derivative for\nthisfunction.\nP′(t) = 4cos3(t) (−sin(t))−sin(\nt4)(\n4t3)\n=−4sin(t)cos3(t)−4t3sin(\nt4)\nThere are a couple of general formulas that we can get for some special cases of the chain rule.\nLet’stakeaquicklookatthose.\nExample 3\nDifferentiateeachofthefollowing.\n(a)f(x) = [g(x)]n\n(b)f(x) =eg(x)\n(c)f(x) =ln(g(x))\nSolution\n(a)f(x) = [g(x)]n\nTheoutsidefunctionistheexponentandtheinsideis g(x).\nf′(x) =

In [19]:
result = agent_executor.invoke({"input": "What is the ratio test and how do you use it to find if the series of (2n)!/(5n+1) converges or diverges?"})



> Entering new AgentExecutor chain...

Invoking: `notes_search` with `{'query': 'ratio test definition and formula'}`


[Document(metadata={'source': 'calculus_notes/Calculus.pdf', 'page': 854, '_id': '73f71f7f4fee48c8a6774e9a626ea592', '_collection_name': 'course_documents'}, page_content='Chapter 10 : Series and Sequences Section 10.10 : Ratio Test\nAgain, we will run across factorials with parenthesis so don’t drop them. This is often one of the\nmorecommonmistakesthatstudentsmakewhentheyfirstrunacrossfactorials.\nOkay,wearenowreadyforthetest.\nRatio Test\nSupposewehavetheseries∑\nan. Define,\nL=lim\nn→∞⏐⏐⏐⏐an+1\nan⏐⏐⏐⏐\nThen,\n1.ifL < 1theseriesisabsolutelyconvergent(andhenceconvergent).\n2.ifL > 1theseriesisdivergent.\n3.ifL= 1theseriesmaybedivergent,conditionallyconvergent,orabsolutelyconver-\ngent.\nAproofofthistestisattheendofthesection.\nNoticethatinthecaseof L= 1theratiotestisprettymuchworthlessandwewouldneedtoresort\ntoadifferenttesttodeterminetheconvergenceoftheseries.\nA

In [24]:
import streamlit as st

if "messages" not in st.session_state:
    st.session_state.messages = []
    
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
        
if prompt := st.chat_input("What is up?"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

# Display assistant response in chat message container
with st.chat_message("assistant"):
    response = st.write_stream(agent_executor.invoke({"input": prompt}))
st.session_state.messages.append({"role": "assistant", "content": response})


2024-08-22 09:49:09.094 Session state does not function when running a script without `streamlit run`




> Entering new AgentExecutor chain...
I don't see a question to solve. Please provide the question you'd like me to assist with, and I'll create a plan to solve it and then solve it in a step-by-step manner using the available tools.

> Finished chain.


In [12]:
from langgraph.checkpoint.memory import MemorySaver  # an in-memory checkpointer
from langgraph.prebuilt import create_react_agent

memory = MemorySaver()
app = create_react_agent(llm, tools, state_modifier=system, checkpointer=memory)
config = {"configurable": {"thread_id": "test-thread"}}

In [55]:
result = app.invoke(
    {
        "messages": [
            ("user", "Solve 2+3")
        ]
    },
    config,
)

In [40]:
result["messages"]

[HumanMessage(content='Solve 2+3', id='d75c4469-22d3-497c-82ba-9e6d9128f92d'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zkcr', 'function': {'arguments': '{"query": "2+3"}', 'name': 'wolfram_call'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 830, 'total_tokens': 848, 'completion_time': 0.072113553, 'prompt_time': 0.221714566, 'queue_time': 0.209402568, 'total_time': 0.293828119}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-556f5b79-4f43-46cd-99f4-820cbe78c985-0', tool_calls=[{'name': 'wolfram_call', 'args': {'query': '2+3'}, 'id': 'call_zkcr', 'type': 'tool_call'}], usage_metadata={'input_tokens': 830, 'output_tokens': 18, 'total_tokens': 848}),
 ToolMessage(content="[[{'title': '', 'plaintext': '2 + 3'}], [{'title': '', 'plaintext': '5'}], [{'title': '', 'plaintext': ''}], [{'title': '', 'plaintext': 'five'}]

In [37]:
result["messages"][-2]

ToolMessage(content="[[{'title': '', 'plaintext': '2 + 3'}], [{'title': '', 'plaintext': '5'}], [{'title': '', 'plaintext': ''}], [{'title': '', 'plaintext': 'five'}], [{'title': '', 'plaintext': ' | + | | = | \\n2 | | 3 | | 5'}], [{'title': '', 'microsources': {'microsource': 'HumanComputationQuery'}, 'plaintext': 'age 6: 4.2 seconds | age 8: 2.2 seconds | age 10: 1.3 seconds | \\nage 18: 0.85 seconds\\n(ignoring concentration, repetition, variations in education, etc.)'}]]", name='wolfram_call', id='feb7eb88-98be-4d05-8b89-bb44d9d5bd4d', tool_call_id='call_bn45')

In [56]:
inputs = {"messages": [("user", "what is the ratio test?")]}


def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

print_stream(app.stream(inputs, stream_mode="values", config=config))

================================ Human Message =================================

what is the ratio test?
================================== Ai Message ==================================
Tool Calls:
  notes_search (call_cc8y)
 Call ID: call_cc8y
  Args:
    query: ratio test definition
================================= Tool Message =================================
Name: notes_search

[Document(metadata={'pk': 451951026610838545, 'source': 'calculus_notes/Calculus.pdf', 'page': 854}, page_content='Chapter 10 : Series and Sequences Section 10.10 : Ratio Test\nAgain, we will run across factorials with parenthesis so don’t drop them. This is often one of the\nmorecommonmistakesthatstudentsmakewhentheyfirstrunacrossfactorials.\nOkay,wearenowreadyforthetest.\nRatio Test\nSupposewehavetheseries∑\nan. Define,\nL=lim\nn→∞⏐⏐⏐⏐an+1\nan⏐⏐⏐⏐\nThen,\n1.ifL < 1theseriesisabsolutelyconvergent(andhenceconvergent).\n2.ifL > 1theseriesisdivergent.\n3.ifL= 1theseriesmaybedivergent,conditionallyconvergent

In [24]:
from IPython.display import display, Latex

display(Latex(result))


<IPython.core.display.Latex object>

In [30]:
inputs2 = {"messages": [("user", "what is the intergral of 3x^2 with respect to x?")]}

print_stream(app.stream(inputs2, stream_mode="values", config=config))

================================ Human Message =================================

what is the intergral of 3x^2 with respect to x?
================================== Ai Message ==================================
Tool Calls:
  wolfram_call (call_48dq)
 Call ID: call_48dq
  Args:
    query: integrate 3x^2 with respect to x
================================= Tool Message =================================
Name: wolfram_call

[[{'title': '', 'plaintext': 'integral3 x^2 dx = x^3 + constant'}], [{'title': '', 'plaintext': ''}]]
================================== Ai Message ==================================

The integral of 3x^2 with respect to x is x^3 + constant.
